In [1]:
import keras

In [2]:
import pandas as pd
import numpy as np
import os,shutil

In [3]:
File_path = './covid-chestxray-dataset-master/metadata.csv' 
Image_path = './covid-chestxray-dataset-master/images/'

In [4]:
df1 = pd.read_csv(File_path)
df1.shape

(950, 30)

In [5]:
df1.head(n=10)

,patientid,offset,sex,age,finding,RT_PCR_positive,survival,intubated,intubation_present,went_icu,...,date,location,folder,filename,doi,url,license,clinical_notes,other_notes,Unnamed: 29
0,2,0.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 22, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-a-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
1,2,3.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 25, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-b-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
2,2,5.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 27, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-c-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
3,2,6.0,M,65.0,Pneumonia/Viral/COVID-19,Y,Y,N,N,N,...,"January 28, 2020","Cho Ray Hospital, Ho Chi Minh City, Vietnam",images,auntminnie-d-2020_01_28_23_51_6665_2020_01_28_...,10.1056/nejmc2001272,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,"On January 22, 2020, a 65-year-old man with a ...",NaN,NaN
4,4,0.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 25, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1a.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,diffuse infiltrates in the bilateral lower lungs,NaN,NaN
5,4,5.0,F,52.0,Pneumonia/Viral/COVID-19,Y,NaN,N,N,N,...,"January 30, 2020","Changhua Christian Hospital, Changhua City, Ta...",images,nejmc2001573_f1b.jpeg,10.1056/NEJMc2001573,https://www.nejm.org/doi/full/10.1056/NEJMc200...,NaN,progressive diffuse interstitial opacities and...,NaN,NaN
6,5,NaN,NaN,NaN,Pneumonia,NaN,NaN,Y,Y,Y,...,2017,NaN,images,ARDSSevere.png,NaN,https://en.wikipedia.org/wiki/File:ARDSSevere.png,CC BY-SA,Severe ARDS. Person is intubated with an OG in...,NaN,NaN
7,6,0.0,NaN,NaN,Pneumonia/Viral/COVID-19,Y,NaN,Y,Y,Y,...,"January 6, 2020","Wuhan Jinyintan Hospital, Wuhan, Hubei Provinc...",images,lancet-case2a.jpg,10.1016/S0140-6736(20)30211-7,https://www.thelancet.com/journals/lancet/arti...,NaN,Case 2: chest x-ray obtained on Jan 6 (2A). Th...,NaN,NaN
8,6,4.0,NaN,NaN,Pneumonia/Viral/COVID-19,Y,NaN,Y,Y,Y,...,"January 10, 2020","Wuhan Jinyintan Hospital, Wuhan, Hubei Provinc...",images,lancet-case2b.jpg,10.1016/S0140-6736(20)30211-7,https://www.thelancet.com/journals/lancet/arti...,NaN,Case 2: chest x-ray obtained on Jan 6 (2A). Th...,NaN,NaN
9,3,4.0,M,74.0,Pneumonia/Viral/SARS,NaN,N,NaN,NaN,NaN,...,2004,"Mount Sinai Hospital, Toronto, Ontario, Canada",images,SARS-10.1148rg.242035193-g04mr34g0-Fig8a-day0....,10.1148/rg.242035193,https://pubs.rsna.org/doi/10.1148/rg.242035193,NaN,SARS in a 74-year-old man who developed sympto...,NaN,NaN


In [6]:
###Finding different type of finding present
np.unique(df1['finding'],return_index=True)

(array(['No Finding', 'Pneumonia', 'Pneumonia/Aspiration',
        'Pneumonia/Bacterial', 'Pneumonia/Bacterial/Chlamydophila',
        'Pneumonia/Bacterial/E.Coli', 'Pneumonia/Bacterial/Klebsiella',
        'Pneumonia/Bacterial/Legionella', 'Pneumonia/Bacterial/Mycoplasma',
        'Pneumonia/Bacterial/Nocardia',
        'Pneumonia/Bacterial/Staphylococcus/MRSA',
        'Pneumonia/Bacterial/Streptococcus',
        'Pneumonia/Fungal/Aspergillosis', 'Pneumonia/Fungal/Pneumocystis',
        'Pneumonia/Lipoid', 'Pneumonia/Viral/COVID-19',
        'Pneumonia/Viral/Herpes ', 'Pneumonia/Viral/Influenza',
        'Pneumonia/Viral/Influenza/H1N1', 'Pneumonia/Viral/MERS-CoV',
        'Pneumonia/Viral/SARS', 'Pneumonia/Viral/Varicella',
        'Tuberculosis', 'Unknown', 'todo'], dtype=object),
 array([ 86,   6, 789, 512, 166, 168, 192, 206, 522, 799, 912,  63, 764,
         61, 505,   0, 765, 538, 755, 809,   9, 507, 752, 419, 730],
       dtype=int64))

In [7]:
###Counting the number of finding with COVID-19 present
cnt = 0
for i in range(df1.shape[0]):
    if 'COVID-19' in df1.iloc[i]['finding'] and df1.iloc[i]['view']=='PA':
        cnt+=1
print(cnt)

196


In [ ]:
#### Moving all covid related data to another directory


Target_folder = './train_dataset/Covid'
if not os.path.exists(Target_folder):
    os.mkdir(Target_folder)
    print('Covid folder created')


In [ ]:
###Moving covid X-rays to another folder
cnt=0
for i in range(df1.shape[0]):
    if 'COVID-19' in df1.iloc[i]['finding'] and df1.iloc[i]['view']=='PA':
        image_path = os.path.join(Image_path,df1.iloc[i]['filename'])
        image_copy_path = os.path.join(Target_folder,df1.iloc[i]['filename'])
        shutil.copy2(image_path,image_copy_path)
        cnt+=1
print(cnt)

In [ ]:
###creating folder of normal x-rays.We will be taking data from the Kaggle dataset

if not os.path.exists('./train_dataset/Normal'):
    os.mkdir('./train_dataset/Normal')
    print('created')



In [ ]:
####We will take only 198 x-rays image from the kaggle to prevent imbalance data
import random
random.seed(5)

In [ ]:
normal_image = os.listdir('./chest_xray/train/NORMAL')
random.shuffle(normal_image)

In [ ]:
for i in range(196):
    img_name = normal_image[i]
    source = os.path.join('./chest_xray/train/NORMAL',img_name)
    dest = os.path.join('./train_dataset/Normal',img_name)
    shutil.copy2(source,dest)

In [ ]:
###Creating directory for storing validation data
if not os.path.exists('./val_dataset'):
    os.mkdir('./val_dataset')
    print('Created')

In [ ]:
if not os.path.exists('./val_dataset/Covid'):
    os.mkdir('./val_dataset/Covid')
    print('created')

if not os.path.exists('./val_dataset/Normal'):
    os.mkdir('./val_dataset/Normal')
    print('created')

In [ ]:
###Moving 20% of training dataset to validation dataset
num = (int)(0.1*len(os.listdir('./train_dataset/Covid')))
cnt=0
for files in os.listdir('./train_dataset/Covid'):
    source = os.path.join('./train_dataset/Covid',files)
    dest = os.path.join('./val_dataset/Covid',files)
    shutil.move(source,dest)
    cnt+=1
    if cnt>=num:
        break
    

In [ ]:
cnt=0
num = (int)(0.1*len(os.listdir('./train_dataset/Normal')))
for files in os.listdir('./train_dataset/Normal'):
    source = os.path.join('./train_dataset/Normal',files)
    dest = os.path.join('./val_dataset/Normal',files)
    shutil.move(source,dest)
    cnt+=1
    if cnt>=num:
        break
    

In [8]:
from keras.preprocessing.image import ImageDataGenerator

In [9]:
###Performing Image Agumentation
train_gen = ImageDataGenerator(
    rescale = 1.0/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

val_gen = ImageDataGenerator(
    rescale = 1.0/255
)

In [10]:
##Creating train generator
train_generator = train_gen.flow_from_directory(
    './train_dataset',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 336 images belonging to 2 classes.


In [11]:
train_generator.class_indices

{'Covid': 0, 'Normal': 1}

In [12]:
##Creating validation generator
val_generator = val_gen.flow_from_directory(
    './val_dataset',
    target_size=(224,224),
    batch_size=32,
    class_mode='binary'
)

Found 55 images belonging to 2 classes.


In [13]:
from keras import Sequential
from keras.layers import *

In [14]:
### Model Architecture
model = Sequential()
model.add(Conv2D(32,(3,3),activation='relu',input_shape=(224,224,3)))
model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.5))

model.add(Conv2D(64,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(128,(3,3),activation='relu'))
model.add(MaxPooling2D((2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(64,activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1,activation='sigmoid'))

model.compile(optimizer='adam',loss=keras.losses.binary_crossentropy,metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 220, 220, 64)      18496     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 110, 110, 64)      0         
_________________________________________________________________
dropout (Dropout)            (None, 110, 110, 64)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 108, 108, 64)      36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 54, 54, 64)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 54, 54, 64)        0

In [15]:
###Fitting the training data
hist = model.fit_generator(
    train_generator,
    steps_per_epoch=8,
    epochs=10,
    validation_data=val_generator,
    validation_steps=2
)

C:\Users\Nishant kumar\Anaconda3\envs\finalattempt\lib\site-packages\tensorflow\python\keras\engine\training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
8/8 [==============================] - 67s 7s/step - loss: 0.8471 - accuracy: 0.5393 - val_loss: 0.6756 - val_accuracy: 0.8545
Epoch 2/10
8/8 [==============================] - 53s 7s/step - loss: 0.6076 - accuracy: 0.7030 - val_loss: 0.4159 - val_accuracy: 0.9273
Epoch 3/10
8/8 [==============================] - 51s 6s/step - loss: 0.4201 - accuracy: 0.7965 - val_loss: 0.2652 - val_accuracy: 0.9818
Epoch 4/10
8/8 [==============================] - 52s 7s/step - loss: 0.3258 - accuracy: 0.8820 - val_loss: 0.1364 - val_accuracy: 0.9818
Epoch 5/10
8/8 [==============================] - 55s 7s/step - loss: 0.1490 - accuracy: 0.9590 - val_loss: 0.1097 - val_accuracy: 0.9818
Epoch 6/10
8/8 [==============================] - 60s 7s/step - loss: 0.2309 - accuracy: 0.9326 - val_loss: 0.1317 - val_accuracy: 0.9636
Epoch 7/10
8/8 [==============================] - 56s 7s/step - loss: 0.1463 - accuracy: 0.9705 - val_loss: 0.0674 - val_accuracy: 0.9636
Epoch 8/10
8/8 [==================

In [16]:
model.evaluate(val_generator)

2/2 [==============================] - 5s 1s/step - loss: 0.1141 - accuracy: 0.9636


[0.11411184072494507, 0.9636363387107849]

In [17]:
model.evaluate(train_generator)

11/11 [==============================] - 34s 3s/step - loss: 0.1997 - accuracy: 0.9762


[0.19974711537361145, 0.976190447807312]